<div style="background-color:rgb(76, 111, 175); padding: 10px; border-radius: 10px;">
    <h1 style="color: Black ; text-align: center; font-weight: bold; font-family: Comic Sans MS, cursive, sans-serif;">
    Arabic Text Summarization
    </h1>
</div>

# Importing Libraries

In [1]:
import re
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Define model , dataset 

In [3]:
base_model ="UBC-NLP/AraT5v2-base-1024"

In [4]:
dataset = load_dataset("FahdSeddik/AGS-Corpus")
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'summary'],
        num_rows: 141467
    })
})

In [5]:
dataset.shape

{'train': (141467, 3)}

# Preprocessing

In [6]:
df = pd.DataFrame({
        'text': dataset['train']['text'],
        'summary': dataset['train']['summary']
    })
df.sample(5)

,text,summary
118079,هذا ما أكده الفنان أحمد جوهر ل ثقافة اليوم بأن...,قرر الفنان أحمد جوهر استعمال فنانين من مصر ولب...
100551,أدانت كوريا الشمالية، الاستخدام المفرط للقوة م...,أدانت كوريا الشمالية استخدام القوة المفرطة من ...
115734,وأضاف التقرير، أن الوصول إلى المياه النظيفة وم...,الوصول إلى المياه النظيفة والنظافة الصحية وغسل...
72730,واختيرت واحدة من بين أجمل عشر نساء فى العالم خ...,تحدث الفنانة ندى بسيوني عن مديحة يسري وأبرز عا...
56588,تعرض خالد للعديد من الانتقادات منذ أسلم، بدءاً...,خالد تعرض للانتقادات بعد اعتناقه الإسلام. وقد ...


In [7]:
print( df['text'][0] )
print(len(df["text"][0]))

يمكن تصور السلوكيات المُهَدِدة باعتبارها ثمرة لعدم القدرة على التأقلم مع الدافع الطبيعي التنافسي المتعلق بعلاقات الهيمنة المتبادلة التي تلاحظ عامة بين الحيوانات. بدلاً من ذلك، قد ينتج الترهيب في مجتمع من نوع يكون أفراده اجتماعيين، فالبشر بشكل عام يترددون في الدخول في مواجهة أو تهديد عنيف.وهو سلوك مثله مثل جميع السمات السلوكية يظهر بشكل أزيد أو أقل في كل فرد مع مرور الزمن، ولكنه قد يكون «سلوك تعويضي» ذو أهمية كبيرة بالنسبة للبعض مقارنة بالآخرين. فإن المنظرين السلوكيين كثيراً ما يرون أن السلوكيات المُهَدِدة هي نتيجة لتعرض القائمين بها للتهديد من قبل الآخرين، بما في ذلك الآباء، ورموز السلطة، والرفاق والأشقاء. «استخدام القوة مبرر عندما يعتقد الشخص بشكل منطقي أنها ضرورية للدفاع عن النفس أو الآخرين تجاه الاستخدام الفوري لقوة غير شرعية».و قد يتم استخدام الترهيب بوعي أو بغير وعي، ونسبة من الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك نتيجة أفكار مستوعبة بأنانية عن تخصيصه لغرض، أو لفائدة أو للتمكين الذاتي. الترهيب المتصل بالتحامل والتمييز يمكن أن يشمل السلوك «الذي يزعج، يهدد، يرهب، وينذر، أو يضع

In [8]:
print( df['summary'][0] )
print(len(df["summary"][0]))

السلوكيات المهددة يمكن أن تكون نتيجة عدم القدرة على التأقلم مع العلاقات الهيمنة التنافسية المتبادلة بين الحيوانات. قد ينتج الترهيب في مجتمع من النوع الاجتماعي. الترهيب يتم استخدامه بوعي أو بغير وعي، ونسبة الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب أفكار أنانية. الترهيب يمكن أن يشمل الاحتكاك البدني، أو التلاعب بالمشاعر، أو الإساءة اللفظية، أو الإحراج المتعمد و/أو الاعتداء الجسدي. قد يشكل تهديد العنف جريمة جنائية.
422


In [9]:
arabic_stopwords = set(stopwords.words('arabic')) 
arabic_stopwords

{'ء',
 'ءَ',
 'آ',
 'آب',
 'آذار',
 'آض',
 'آمينَ',
 'آناء',
 'آنفا',
 'آه',
 'آها',
 'آهاً',
 'آهٍ',
 'آهِ',
 'آي',
 'أ',
 'أبدا',
 'أبريل',
 'أبو',
 'أبٌ',
 'أجل',
 'أجمع',
 'أحد',
 'أخبر',
 'أخذ',
 'أخو',
 'أخٌ',
 'أربع',
 'أربعاء',
 'أربعة',
 'أربعمئة',
 'أربعمائة',
 'أرى',
 'أسكن',
 'أصبح',
 'أصلا',
 'أضحى',
 'أطعم',
 'أعطى',
 'أعلم',
 'أغسطس',
 'أف',
 'أفريل',
 'أفعل به',
 'أفٍّ',
 'أقبل',
 'أقل',
 'أكتوبر',
 'أكثر',
 'أل',
 'ألا',
 'ألف',
 'ألفى',
 'أم',
 'أما',
 'أمام',
 'أمامك',
 'أمامكَ',
 'أمد',
 'أمس',
 'أمسى',
 'أمّا',
 'أن',
 'أنا',
 'أنبأ',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'أنتِ',
 'أنشأ',
 'أنى',
 'أنًّ',
 'أنّى',
 'أهلا',
 'أو',
 'أوت',
 'أوشك',
 'أول',
 'أولئك',
 'أولاء',
 'أولالك',
 'أوه',
 'أوّهْ',
 'أى',
 'أي',
 'أيا',
 'أيار',
 'أيضا',
 'أيلول',
 'أين',
 'أينما',
 'أيها',
 'أيّ',
 'أيّان',
 'أُفٍّ',
 'ؤ',
 'إحدى',
 'إذ',
 'إذا',
 'إذاً',
 'إذما',
 'إذن',
 'إزاء',
 'إلا',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'إليكنّ',
 'إليكَ',
 'إلَيْكَ',
 'إلّا',


### Normalize Text

In [10]:
arabic_patterns = {
            # Remove diacritics
            r'[\u064B-\u065F\u0670]': '',
            # Normalize Arabic characters
            r'[إأآا]': 'ا',
            r'ى': 'ي',
            r'ة': 'ه'
        }

In [11]:
def normalize_arabic(text):
        for pattern, replacement in arabic_patterns.items():
            text = re.sub(pattern, replacement, text)
        return text

### Clean the words 
the Arabic stop words we load earlier and the text 

In [12]:
def clean_stopwords(stopwords_set):
    cleaned_stopwords = set()
    for word in stopwords_set:
        # normalize them 
        word = normalize_arabic(word)
        cleaned_stopwords.add(word)
    return cleaned_stopwords

# cleaned set 
arabic_stopwords = clean_stopwords(arabic_stopwords)

In [13]:
def clean_text(text):
        # remove whitespace
        text = re.sub(r'\s+', ' ', text)
        # remove  urls
        text = re.sub(r'http\S+|www.\S+', '', text)
        # remove emails
        text = re.sub(r'\S+@\S+', '', text)
        return text.strip()

In [14]:
def remove_punctuation(text):
        arabic_punctuation = '،؛؟'
        # remove other punctuation 
        punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        
        punctuation_to_remove = ''.join(ch for ch in punctuation if ch not in arabic_punctuation)
        return text.translate(str.maketrans('', '', punctuation_to_remove))

### Custom Tokenization 

In [15]:
def ara_tokenizer(text):
        text = normalize_arabic(text)
        tokens = re.findall(r'\w+', text)
        return [token for token in tokens if token not in arabic_stopwords]

# Feature Extraction 

we will extract the basic features for any text

In [16]:
def extract_features(text):
    if not isinstance(text, str) or not text.strip():
        return {
            'length': 0,
            'word_count': 0,
            'avg_word_length': 0,
            'lexical_diversity': 0
        }
    
    text = normalize_arabic(text)  
    tokens = ara_tokenizer(text)

    features = {
        'length': len(text),
        'word_count': len(tokens),
        'avg_word_length': sum(len(word) for word in tokens) / max(1, len(tokens)),
        'lexical_diversity': len(set(tokens)) / max(1, len(tokens))
    }
    
    return features

**Combining all the function in one to implement preprocessing**

In [17]:
def preprocess_text(text):
        # check if its a text 
        if not isinstance(text, str):
            return ""
            
        text = str(text).strip()
        
        text = clean_text(text)
        text = remove_punctuation(text)
        text = normalize_arabic(text)
            
        return text

### Define TF_IDF 

just an extra step to implement cosain similarty 

In [18]:
tfidf_vectorizer = TfidfVectorizer(
            max_features=500, 
            ngram_range=(1, 2),
            tokenizer= lambda text: ara_tokenizer(text),
            stop_words=None
        )

## implement on the dataset

In [19]:
# implementing to df_with_features_with_features text and summary 
df['text'] = df['text'].apply(preprocess_text) 
df['summary'] = df['summary'].apply(preprocess_text)  
df['text'][0]

'يمكن تصور السلوكيات المهدده باعتبارها ثمره لعدم القدره علي التاقلم مع الدافع الطبيعي التنافسي المتعلق بعلاقات الهيمنه المتبادله التي تلاحظ عامه بين الحيوانات بدلا من ذلك، قد ينتج الترهيب في مجتمع من نوع يكون افراده اجتماعيين، فالبشر بشكل عام يترددون في الدخول في مواجهه او تهديد عنيفوهو سلوك مثله مثل جميع السمات السلوكيه يظهر بشكل ازيد او اقل في كل فرد مع مرور الزمن، ولكنه قد يكون «سلوك تعويضي» ذو اهميه كبيره بالنسبه للبعض مقارنه بالاخرين فان المنظرين السلوكيين كثيرا ما يرون ان السلوكيات المهدده هي نتيجه لتعرض القائمين بها للتهديد من قبل الاخرين، بما في ذلك الاباء، ورموز السلطه، والرفاق والاشقاء «استخدام القوه مبرر عندما يعتقد الشخص بشكل منطقي انها ضروريه للدفاع عن النفس او الاخرين تجاه الاستخدام الفوري لقوه غير شرعيه»و قد يتم استخدام الترهيب بوعي او بغير وعي، ونسبه من الاشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك نتيجه افكار مستوعبه بانانيه عن تخصيصه لغرض، او لفائده او للتمكين الذاتي الترهيب المتصل بالتحامل والتمييز يمكن ان يشمل السلوك «الذي يزعج، يهدد، يرهب، وينذر، او يضع الشخص في حال

In [20]:
df['summary'][0]

'السلوكيات المهدده يمكن ان تكون نتيجه عدم القدره علي التاقلم مع العلاقات الهيمنه التنافسيه المتبادله بين الحيوانات قد ينتج الترهيب في مجتمع من النوع الاجتماعي الترهيب يتم استخدامه بوعي او بغير وعي، ونسبه الاشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب افكار انانيه الترهيب يمكن ان يشمل الاحتكاك البدني، او التلاعب بالمشاعر، او الاساءه اللفظيه، او الاحراج المتعمد واو الاعتداء الجسدي قد يشكل تهديد العنف جريمه جنائيه'

In [21]:
# comparison
before="السلوكيات المهددة يمكن أن تكون نتيجة عدم القدرة على التأقلم مع العلاقات الهيمنة التنافسية المتبادلة بين الحيوانات. قد ينتج الترهيب في مجتمع من النوع الاجتماعي. الترهيب يتم استخدامه بوعي أو بغير وعي، ونسبة الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب أفكار أنانية. الترهيب يمكن أن يشمل الاحتكاك البدني، أو التلاعب بالمشاعر، أو الإساءة اللفظية، أو الإحراج المتعمد و/أو الاعتداء الجسدي. قد يشكل تهديد العنف جريمة جنائية"
after ="السلوكيات المهدده يمكن ان تكون نتيجه عدم القدره علي التاقلم مع العلاقات الهيمنه التنافسيه المتبادله بين الحيوانات قد ينتج الترهيب في مجتمع من النوع الاجتماعي الترهيب يتم استخدامه بوعي او بغير وعي، ونسبه الاشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب افكار انانيه الترهيب يمكن ان يشمل الاحتكاك البدني، او التلاعب بالمشاعر، او الاساءه اللفظيه، او الاحراج المتعمد واو الاعتداء الجسدي قد يشكل تهديد العنف جريمه جنائيه"

In [23]:
# extract the features
text_features = pd.DataFrame(df['text'].apply(extract_features).tolist())
text_features.columns = [f'text_{col}' for col in text_features.columns]

summary_features = pd.DataFrame(df['summary'].apply(extract_features).tolist())
summary_features.columns = [f'summary_{col}' for col in summary_features.columns]

# combine the data with the extracted features by column
df_with_features = pd.concat([df, text_features, summary_features], axis=1)



In [24]:
df_with_features.head()

,text,summary,text_length,text_word_count,text_avg_word_length,text_lexical_diversity,summary_length,summary_word_count,summary_avg_word_length,summary_lexical_diversity
0,يمكن تصور السلوكيات المهدده باعتبارها ثمره لعد...,السلوكيات المهدده يمكن ان تكون نتيجه عدم القدر...,1850,220,5.663636,0.877273,416,50,6.020000,0.920000
1,يقدر المؤرخون مجموع عدد الرقيق خلال الفتره الم...,يقدر عدد الرقيق خلال الفتره الممتده من 650 الي...,1679,206,5.626214,0.844660,487,60,5.683333,0.950000
2,الرياض قامت صباح امس السبت بجوله في سوق السبت ...,قامت الرياض صباح امس السبت بجوله في سوق السبت ...,965,120,5.366667,0.783333,281,36,5.750000,0.972222
3,كان السميلودون اخر اجناس السنوريات سيفيه الاني...,السميلودون هو اخر اجناس السنوريات سيفيه الانيا...,1549,187,6.363636,0.716578,421,49,6.795918,0.918367
4,وقال سموه في كلمه له بمناسبه اليوم الوطني ان ك...,لم يكن اليوم الوطني يعتبر تحقيقا لولا مسيره ال...,847,113,5.433628,0.734513,361,49,5.714286,0.897959


## Split the Dataset

In [25]:
# Shuffle the dataframe
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(len(df) * 0.8)  
val_size = int(len(df) * 0.1)    
test_size = len(df) - (train_size + val_size)  

# Split the dataframe
train_df = df[:train_size]
val_df = df[train_size:train_size + val_size]
test_df = df[train_size + val_size:]

print(len(train_df) , len(val_df) ,len(test_df) )


113173 14146 14148


In [ ]:
df.info()
print("\n\n")
df_with_features.info()

**Save them in CSV files**

create directory for Data before

In [27]:
data = "data"
train_df.to_csv(f'{data}/train.csv', index=False)
val_df.to_csv(f'{data}/val.csv', index=False)
test_df.to_csv(f'{data}/test.csv', index=False)

In [28]:
from datasets import Dataset

### convert to HF dataset 

for Speed, memory, Compatibility and easy Tokenization

In [ ]:
train_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/val.csv")
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)# convert to HF for 

In [29]:
train_df = pd.read_csv("data/train.csv")
train_df = train_df.sample(n=30000, random_state=42)

In [30]:
val_df = pd.read_csv("data/val.csv")
val_df = val_df.sample(n=6000, random_state=42)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)# convert to HF for 

# Model 

**import libraries**

In [31]:
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, 
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    DataCollatorForSeq2Seq
)
import evaluate
from torch.utils.checkpoint import checkpoint_sequential
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
from peft import LoraConfig, PeftModel, get_peft_model

We use 4-bit quantization to reduce the model size and memory usage during training or inference, making it faster and more efficient.


In [33]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

## Load Model and Tokenizer

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(
    base_model,
    quantization_config=quant_config
).to(device)

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.model_max_length = 512

model.config.use_cache = False
model.config.pretraining_tp = 1

`low_cpu_mem_usage` was None, now default to True since model is quantized.


## LORA Config
LoRA (Low-Rank Adaptation) is used to fine-tune large models efficiently by updating only a small number of parameters instead of the full model weights, reducing memory and compute requirements.

Here, we define a `LoraConfig` to specify how LoRA will be applied, including settings like the rank, dropout, and task type.

This configuration will be passed later when setting up the model for training or fine-tuning.

In [35]:
# lora config
lora_config  = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

In [36]:
model = get_peft_model(model, lora_config)

## Tokenization Function 

In [37]:
def tokenize(examples):
    # prefix summarize to the input text for AraT5
    inputs = ["summarize: " + text for text in examples["text"]]
    summaries = examples["summary"]

    model_inputs = tokenizer(
        inputs,
        max_length=512,
        truncation=True,
        padding=False
    )
    labels = tokenizer(
        summaries,
        max_length=128,
        truncation=True,
        padding=False
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


**using DataCollator for Dynamic padding. Rather than a fixed lenght for all Seqs, it would be the longest Seq in the batch**

In [38]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=base_model, padding="longest", return_tensors="pt")

## Compute Metrics for Model Evaluation



The function calculates various evaluation metrics such as Rouge and Cosine Similarity to assess the performance of the model in generating summaries

In [39]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # replace -100 with pad_token_id for decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # decoding the predicted
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    preprocessed_preds = [preprocess_text(text) for text in decoded_preds]
    preprocessed_labels = [preprocess_text(text) for text in decoded_labels]

    # rouge scores 
    rouge = evaluate.load("rouge")
    rouge_results = rouge.compute(
        predictions=preprocessed_preds,
        references=preprocessed_labels,
        use_stemmer=True
    )
    
    # TF-IDF Vectorization for cosine similarity
    eval_vectorizer = TfidfVectorizer(
        max_features=500,
        ngram_range=(1, 2),
        tokenizer=lambda text: ara_tokenizer(text) if text else [],
        stop_words=None
    )
    
    # apply to predictions and labels 
    pred_vectors = eval_vectorizer.fit_transform(preprocessed_preds)
    label_vectors = eval_vectorizer.transform(preprocessed_labels)
    
    # calc cosine similarity
    cosine_similarities = []
    for i in range(len(preprocessed_preds)):
        # check it the vector is not empty
        if pred_vectors[i].nnz > 0 and label_vectors[i].nnz > 0:
            similarity = cosine_similarity(pred_vectors[i], label_vectors[i])[0][0]
            # Guard against NaN or infinite values
            if np.isfinite(similarity):
                cosine_similarities.append(similarity)
            else:
                cosine_similarities.append(0.0)
        else:
            cosine_similarities.append(0.0)
            
    avg_cosine_sim = np.mean(cosine_similarities) if cosine_similarities else 0.0

    metrics = {
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "cosine_similarity": avg_cosine_sim
    }

    return metrics

## Tokenize the data

In [40]:
tokenized_train = train_dataset.map(
    tokenize,
    batched=True,
    remove_columns=train_dataset.column_names
)
tokenized_val = val_dataset.map(
    tokenize,
    batched=True,
    remove_columns=val_dataset.column_names
)


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

## Train the model

In [70]:
# training args
training_args = Seq2SeqTrainingArguments(
    output_dir='models',
    num_train_epochs=2,
    learning_rate=5e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,         
    gradient_accumulation_steps=1,
    save_steps=500,
    logging_steps=100,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine" ,
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,  # Perform evaluation every 500 steps
    save_total_limit=1,  # Save only the best model
    metric_for_best_model="rouge2",
    greater_is_better=True,
    predict_with_generate=True  # Generate predictions during evaluation
)


c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [76]:
from transformers import TrainerCallback , EarlyStoppingCallback


class DynamicEvalDatasetCallback(TrainerCallback):
    def __init__(self, full_eval_dataset, trainer):
        self.full_eval_dataset = full_eval_dataset
        self.trainer = trainer

    def on_step_begin(self, args, state, control, **kwargs):
        if state.global_step % args.eval_steps == 0 and state.global_step != 0:
            small_val_dataset = self.full_eval_dataset.shuffle(seed=state.global_step).select(range(500))
            self.trainer.eval_dataset = small_val_dataset



In [73]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=17
)

In [84]:

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val.select(range(500)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
    data_collator=data_collator,
    callbacks=[
        dynamic_eval_callback,  
        early_stopping_callback                      
    ],
)


C:\Users\ACER\AppData\Local\Temp\ipykernel_6584\652832434.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [83]:
dynamic_eval_callback = DynamicEvalDatasetCallback(full_eval_dataset=tokenized_val, trainer=trainer)

In [43]:
torch.cuda.empty_cache()

In [ ]:
# Update training arguments to include evaluation during training
training_args = Seq2SeqTrainingArguments(
    output_dir='models',
    num_train_epochs=2,
    learning_rate=1e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    save_steps=0,
    logging_steps=25,
    weight_decay=0.001,
    fp16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=100,  # Perform evaluation every 100 steps
    save_total_limit=1,  # Save only the best model
    metric_for_best_model="rouge2",
    greater_is_better=True,
    predict_with_generate=True,  # Generate predictions during evaluation
)

# Initialize the trainer with updated arguments
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

# Train and evaluate
trainer.train()

In [85]:
trainer.train()

Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Cosine Similarity
500,3.227500,2.260849,0.039032,0.008131,0.038570,0.349898
1000,3.130000,2.259577,0.037384,0.008972,0.037228,0.346257
1500,3.299000,2.246920,0.038490,0.008052,0.037771,0.349770
2000,3.162500,2.250000,0.040196,0.008724,0.039233,0.345411
2500,3.149200,2.228877,0.040465,0.008553,0.040716,0.349918
3000,3.159100,2.232502,0.041172,0.010046,0.040085,0.346453
3500,3.129100,2.220701,0.041236,0.008644,0.040762,0.346785
4000,3.147700,2.211735,0.041107,0.010299,0.039001,0.348830
4500,3.173100,2.207544,0.039571,0.010197,0.038650,0.356687
5000,3.181000,2.201705,0.039691,0.010162,0.038277,0.348396


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\peft\utils\other.py:1107: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 20034c79-bc3b-4da5-91bf-9ca6ac5e4d6e)') - silently ignoring the lookup for the file config.json in UBC-NLP/AraT5v2-base-1024.
  warnings.warn(
c:\Users\ACER\anaconda3\envs\llama\lib\site-packages\peft\utils\save_and_load.py:236: UserWarning: Could not find a config file in UBC-NLP/AraT5v2-base-1024 - will assume that the vocabulary was not modified.
  wa

TrainOutput(global_step=12500, training_loss=3.1733781103515626, metrics={'train_runtime': 20690.0166, 'train_samples_per_second': 2.9, 'train_steps_per_second': 0.725, 'total_flos': 2.158002803902464e+16, 'train_loss': 3.1733781103515626, 'epoch': 1.6666666666666665})

**output for the first training without eval in training**
TrainOutput(global_step=15000, training_loss=3.7461526138305663, metrics={'train_runtime': 4475.4542, 'train_samples_per_second': 13.406, 'train_steps_per_second': 3.352, 'total_flos': 2.588429430864691e+16, 'train_loss': 3.7461526138305663, 'epoch': 2.0})


# Model Evaluation

In [57]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


KeyboardInterrupt: 

# Save the model

# Generate Summary

In [ ]:
generation_config = {
    "max_length": 128,
    "min_length": 30,
    "num_beams": 5,
    "length_penalty": 1.5,
    "early_stopping": True
}


In [ ]:
def generate_summary(text):
    input_text = "summarize: " + preprocess_text(text)
    
    inputs = tokenizer(
        input_text,
        max_length=512,
        truncation=True,
        padding=False,
        return_tensors="pt"
    ).to(model.device)
    
    # generate 
    summary_ids = model.generate(
        inputs["input_ids"],
        **generation_config
    )
    
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)